In [5]:
import re
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup as bs
import time
import sqlite3
from urllib.parse import quote_plus
import yaml
from datetime import datetime
import os
from KBDownloader import search_swedish_newspapers, fetch_newspaper_data, save_checkpoint, load_checkpoint

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

# Load the YAML configuration file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Assign variables from the YAML configuration
venue_list = config['venue_list']
start_year = config['start_year']
years_to_crawl = config['years_to_crawl']
newspaper = config['newspaper']
db_path = config['db_path']

# Define the newspaper collection IDs
NEWSPAPER_COLLECTION_IDS = {
    'Dagens nyheter': 'https://libris.kb.se/m5z2w4lz3m2zxpk#it',
    'Svenska Dagbladet': 'https://libris.kb.se/2ldhmx8d4mcrlq9#it',
    'Aftonbladet': 'https://libris.kb.se/dwpgqn5q03ft91j#it',
    'Dagligt Allehanda': 'https://libris.kb.se/9tmqzv3m32xfzcz#it'
}

# Get the correct collection ID for the specified newspaper
collection_id = NEWSPAPER_COLLECTION_IDS.get(newspaper)
if not collection_id:
    raise ValueError(f"Invalid newspaper name: {newspaper}")

# Ensure the database file exists
if not os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    conn.close()

# Load checkpoint if it exists
checkpoint = load_checkpoint()
start_year = checkpoint['year'] if checkpoint else start_year
start_half = checkpoint['half'] if checkpoint else 0
start_index = checkpoint['index'] if checkpoint else 0

# Print out all the settings from the YAML configuration file
print("Configuration Settings:")
for key, value in config.items():
    print(f"{key}: {value}")
print(f"Collection ID: {collection_id}")

# Load the venue list
df = pd.read_excel(venue_list)

Configuration Settings:
venue_list: Datasets/Venues_plus_Konsert.xlsx
start_year: 1908
years_to_crawl: 1
newspaper: Dagens nyheter
prompt_filepath: llm_prompt_for_deployment.txt
db_path: Datasets/21.07_Deployment_Version_4.db
llm_model: gpt-4o-mini
max_tokens: 1000
Stockholm_Concert_Database_Path: Datasets/All_Concerts_1908_filtered.xlsx
columns_to_compare: ['normalized_date', 'normalized_konsert_namn', 'lokal_namn']
Collection ID: https://libris.kb.se/m5z2w4lz3m2zxpk#it


In [8]:
# Main loop
for year in range(start_year, start_year + years_to_crawl):
    for half in range(start_half, 2):
        if half == 0:
            from_date = datetime(year, 1, 1)
            to_date = datetime(year, 6, 30)
        else:
            from_date = datetime(year, 7, 1)
            to_date = datetime(year, 12, 31)

        for index in range(start_index, len(df)):
            row = df.iloc[index]
            # ... (data fetching and processing code)
            
            # Save checkpoint after each query, successful or not
            save_checkpoint(year, half, index + 1)
            
        print(f"Waiting. Currently at {from_date} to {to_date}")
        time.sleep(3) # in seconds
        start_index = 0  # Reset start_index for the next half-year

    start_half = 0  # Reset start_half for the next year



# Main loop
for year in range(start_year, start_year + years_to_crawl):
    for half in range(start_half, 2):
        if half == 0:
            from_date = datetime(year, 1, 1)
            to_date = datetime(year, 6, 30)
        else:
            from_date = datetime(year, 7, 1)
            to_date = datetime(year, 12, 31)

        print(f"Processing data from {from_date} to {to_date}")

        for index in range(start_index, len(df)):
            row = df.iloc[index]
            query = row['Lokal']

            try:
                result = fetch_newspaper_data(
                    query=query,
                    from_date=from_date.strftime('%Y-%m-%d'),
                    to_date=to_date.strftime('%Y-%m-%d'),
                    newspaper=collection_id,
                    config=config,
                    db_path=db_path
                )

                if result.get('success'):
                    print(f"Processed query '{query}' successfully.")
                else:
                    print(f"Failed to process query '{query}': {result.get('message')}")

                # Save checkpoint after each query, successful or not
                save_checkpoint(year, half, index + 1)

            except Exception as e:
                print(f"Error processing query '{query}': {str(e)}")
                save_checkpoint(year, half, index)
                # Consider raising specific exceptions here instead of a broad Exception

        print(f"Waiting. Currently at {from_date} to {to_date}")
        time.sleep(3)  # in seconds

    start_half = 0  # Reset start_half for the next year

print("All queries processed for all years")



2024-07-22 16:14:52,927 - INFO - Starting fetch_newspaper_data for query: Konsert, dates: 1908-01-01 to 1908-06-30
2024-07-22 16:14:53,217 - INFO - Search results received. Hits: 20
2024-07-22 16:14:53,217 - INFO - Extracted 20 URLs from search results
2024-07-22 16:14:53,220 - INFO - Table 'newspaper_data' created or already exists
2024-07-22 16:14:53,221 - INFO - Processing URL: https://data.kb.se/dark-3693573/part/1/page/4
2024-07-22 16:14:53,355 - INFO - Extracted 1 XML URLs
2024-07-22 16:14:53,642 - INFO - Fetched XML content for 1 pages
2024-07-22 16:14:53,836 - INFO - Inserted or updated row 1 in database
2024-07-22 16:14:53,837 - INFO - Inserted or updated row 2 in database
2024-07-22 16:14:53,838 - INFO - Inserted or updated row 3 in database
2024-07-22 16:14:53,839 - INFO - Inserted or updated row 4 in database
2024-07-22 16:14:53,839 - INFO - Inserted or updated row 5 in database
2024-07-22 16:14:53,840 - INFO - Inserted or updated row 6 in database
2024-07-22 16:14:53,840 -

Processed query 'Konsert' successfully.
Checkpoint saved: Year 1908, Half 0, Index 1
Processed query 'La Croix salong' successfully.
Checkpoint saved: Year 1908, Half 0, Index 2


2024-07-22 16:15:07,794 - INFO - Search results received. Hits: 0
2024-07-22 16:15:07,795 - INFO - Extracted 0 URLs from search results
2024-07-22 16:15:07,797 - INFO - Table 'newspaper_data' created or already exists
2024-07-22 16:15:07,798 - INFO - Data processing completed. Total rows saved: 0
2024-07-22 16:15:07,800 - INFO - Starting fetch_newspaper_data for query: Wallmans lokal (Mäster Samuels gränd 11), dates: 1908-01-01 to 1908-06-30


Processed query 'Norra paviljongen i Trädgårdsföreningens lokal' successfully.
Checkpoint saved: Year 1908, Half 0, Index 3


2024-07-22 16:15:08,220 - INFO - Search results received. Hits: 0
2024-07-22 16:15:08,222 - INFO - Extracted 0 URLs from search results
2024-07-22 16:15:08,223 - INFO - Table 'newspaper_data' created or already exists
2024-07-22 16:15:08,225 - INFO - Data processing completed. Total rows saved: 0
2024-07-22 16:15:08,227 - INFO - Starting fetch_newspaper_data for query: Kungliga opera, dates: 1908-01-01 to 1908-06-30


Processed query 'Wallmans lokal (Mäster Samuels gränd 11)' successfully.
Checkpoint saved: Year 1908, Half 0, Index 4


2024-07-22 16:15:08,639 - INFO - Search results received. Hits: 20
2024-07-22 16:15:08,640 - INFO - Extracted 20 URLs from search results
2024-07-22 16:15:08,642 - INFO - Table 'newspaper_data' created or already exists
2024-07-22 16:15:08,643 - INFO - Processing URL: https://data.kb.se/dark-3693840/part/1/page/2
2024-07-22 16:15:08,790 - INFO - Extracted 1 XML URLs
2024-07-22 16:15:09,055 - INFO - Fetched XML content for 1 pages
2024-07-22 16:15:09,240 - INFO - Inserted or updated row 1 in database
2024-07-22 16:15:09,242 - INFO - Inserted or updated row 2 in database
2024-07-22 16:15:09,243 - INFO - Inserted or updated row 3 in database
2024-07-22 16:15:09,246 - INFO - Committed changes for URL: https://data.kb.se/dark-3693840/part/1/page/2
2024-07-22 16:15:09,247 - INFO - Processing URL: https://data.kb.se/dark-3693618/part/1/page/1
2024-07-22 16:15:09,434 - INFO - Extracted 1 XML URLs
2024-07-22 16:15:09,726 - INFO - Fetched XML content for 1 pages
2024-07-22 16:15:10,055 - INFO - 

# View SQL Database

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# SQL query to select all columns except 'Raw API Result'
query = """
SELECT Date, [Package ID], Part, Page, [ComposedBlock Content], [Full Prompt]
FROM newspaper_data
"""

# Read the query results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
print(df.head())

# Display basic information about the DataFrame
print(df.info())

# Optional: If you want to see all rows, you can use:
# pd.set_option('display.max_rows', None)
# print(df)

# Optional: If you want to save this to a CSV file for further analysis:
# df.to_csv('newspaper_data_summary.csv', index=False)